In [10]:
import pandas as pd
from transformers import pipeline
from nltk.translate.bleu_score import corpus_bleu
from nltk.tokenize import word_tokenize
from nltk.util import ngrams
from math import exp
from tqdm import tqdm 

In [11]:
train_df1 = pd.read_csv("C:/Users/Abhinandan Sharma/Desktop/topsis_asg/train.csv")
train_df=train_df1.head(150)

In [22]:
models = [
    "facebook/bart-large-cnn",
    "t5-large",
    "sshleifer/distilbart-cnn-12-6",
    "google/pegasus-large",
    "allenai/led-large-16384-arxiv",
    "sshleifer/bart-tiny-random",
  
]

In [13]:
def semantic_coherence(generated_summary, dialogue):
    summary_tokens = word_tokenize(generated_summary.lower())
    dialogue_tokens = word_tokenize(dialogue.lower())
    common_tokens = set(summary_tokens) & set(dialogue_tokens)
    coherence_score = len(common_tokens) / len(summary_tokens)
    return coherence_score


In [14]:
def factual_accuracy(generated_summary, reference_summary):
    gen_tokens = set(word_tokenize(generated_summary.lower()))
    ref_tokens = set(word_tokenize(reference_summary.lower()))
    common_tokens = gen_tokens & ref_tokens
    accuracy_score = len(common_tokens) / len(ref_tokens) if len(ref_tokens) != 0 else 0
    return accuracy_score


In [15]:
def content_coverage(generated_summary, dialogue):
    summary_tokens = set(word_tokenize(generated_summary.lower()))
    dialogue_tokens = set(word_tokenize(dialogue.lower()))
    common_tokens = summary_tokens & dialogue_tokens
    coverage_score = len(common_tokens) / len(dialogue_tokens) if len(dialogue_tokens) != 0 else 0
    return coverage_score


In [16]:
from math import log
def fluency(generated_summary):
    tokens = word_tokenize(generated_summary.lower())
    token_freq = Counter(tokens)
    total_log_prob = sum(-log(token_freq[token] / len(tokens)) for token in token_freq)
    perplexity = exp(total_log_prob / len(tokens))
    return perplexity

In [17]:
evaluation_results_list = []

In [19]:
import nltk
from nltk.tokenize import word_tokenize

# Download the 'punkt' resource
nltk.download('punkt')



[nltk_data] Downloading package punkt to C:\Users\Abhinandan
[nltk_data]     Sharma\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


True

In [20]:
from collections import Counter


In [23]:
for model_name in models:
    print(f"Evaluating model: {model_name}")
    summarizer = pipeline("summarization", model=model_name, tokenizer=model_name)
    semantic_coherence_scores = []
    factual_accuracy_scores = []
    content_coverage_scores = []
    fluency_scores = []
    generated_summaries = []
    for index, row in tqdm(train_df.iterrows(), total=len(train_df)):  # Use tqdm to show progress
        dialogue = row['dialogue']
        summary = row['summary']
        generated_summary = summarizer(dialogue, max_length=150, min_length=30, do_sample=False)[0]["summary_text"]
        generated_summaries.append(generated_summary)
        coherence_score = semantic_coherence(generated_summary, dialogue)
        semantic_coherence_scores.append(coherence_score)
        accuracy_score = factual_accuracy(generated_summary, summary)
        factual_accuracy_scores.append(accuracy_score)
        coverage_score = content_coverage(generated_summary, dialogue)
        content_coverage_scores.append(coverage_score)
        fluency_score = fluency(generated_summary)
        fluency_scores.append(fluency_score)
    evaluation_results_list.append({
        "Model": model_name,
        "Semantic Coherence": sum(semantic_coherence_scores) / len(semantic_coherence_scores),
        "Factual Accuracy": sum(factual_accuracy_scores) / len(factual_accuracy_scores),
        "Content Coverage": sum(content_coverage_scores) / len(content_coverage_scores),
        "Fluency": sum(fluency_scores) / len(fluency_scores)
    })
    print("="*50)

Evaluating model: facebook/bart-large-cnn


 55%|████████████████████████████████████████████▎                                    | 82/150 [10:03<08:20,  7.35s/it]Your max_length is set to 150, but your input_length is only 85. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=42)


100%|████████████████████████████████████████████████████████████████████████████████| 150/150 [19:20<00:00,  7.74s/it]


Evaluating model: t5-large


E:\anaconda\Lib\site-packages\transformers\models\t5\tokenization_t5_fast.py:155: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-large automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(
 68%|██████████████████████████████████████████████████████▍                         | 102/150 [30:31<16:31, 20.67s/it]Your max_length is set to 150, but your input_length is only 143. Since this is a summarization task, where outputs shorter than the 

...

 55%|██████████████████████████████████████████                                   | 82/150 [2:06:12<1:35:19, 84.12s/it]Your max_length is set to 150, but your input_length is only 85. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=42)


100%|██████████████████████████████████████████████████████████████████████████████| 150/150 [3:58:31<00:00, 95.41s/it]


Evaluating model: sshleifer/bart-tiny-random


  0%|                                                                                          | 0/150 [00:00<?, ?it/s]E:\anaconda\Lib\site-packages\transformers\generation\configuration_utils.py:399: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
E:\anaconda\Lib\site-packages\transformers\generation\configuration_utils.py:399: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(
 51%|█████████████████████████████████████████▌                                       | 77/150 [00:46<00:46,  1.57it/s]Your max_lengt

100%|████████████████████████████████████████████████████████████████████████████████| 150/150 [01:30<00:00,  1.66it/s]

In [24]:
print(len(evaluation_results_list))

7


In [25]:
result = pd.concat([pd.DataFrame(item, index=[0]) for item in evaluation_results_list], ignore_index=True)

In [26]:
result.to_csv("result.csv", index=False)

In [28]:
result.head(6)

,Model,Semantic Coherence,Factual Accuracy,Content Coverage,Fluency
0,facebook/bart-large-cnn,0.655644,0.458349,0.371606,13.815770
1,facebook/bart-large-cnn,0.655644,0.458349,0.371606,13.815770
2,t5-large,0.754954,0.457265,0.409083,15.751138
3,sshleifer/distilbart-cnn-12-6,0.707827,0.521906,0.484371,15.687915
4,google/pegasus-large,0.706821,0.420956,0.364438,11.779998
5,allenai/led-large-16384-arxiv,0.170011,0.292440,0.190508,3.732631
